In [1]:
# Michael Spearing
# Started: April 26, 2017
# Data Science Lab

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pygame
import time
pygame.mixer.init()
pygame.mixer.music.load("beep-07.wav")


In [3]:
def alert(num = 1):
    for x in range(num):
        pygame.mixer.music.play()
        time.sleep(2)

In [4]:
# Adding & preprocessing the weather data
# SOURCE: https://www.ncdc.noaa.gov/
file_weather = "WeatherData.csv"
data_weather_full = pd.read_csv(file_weather, skipinitialspace = True)
features_weather = ["YEARMODA", "TEMP", "PRCP"]
data_weather = data_weather_full.loc[:,features_weather]
print data_weather.head()

for i in range(len(data_weather)):
    tmp = np.float64(data_weather.loc[i,"PRCP"][:4])
    data_weather.loc[i,"PRCP"] = tmp
print(data_weather.head())
for ft in features_weather:
    print(type(data_weather.loc[1,ft]))

   YEARMODA  TEMP   PRCP
0  20140101  49.1  0.00G
1  20140102  47.5  0.00G
2  20140103  40.6  0.00G
3  20140104  53.8  0.00G
4  20140105  51.8  0.00G
   YEARMODA  TEMP PRCP
0  20140101  49.1    0
1  20140102  47.5    0
2  20140103  40.6    0
3  20140104  53.8    0
4  20140105  51.8    0
<type 'numpy.int64'>
<type 'numpy.float64'>
<type 'numpy.float64'>


In [5]:
dates_weather = []
for i in range(len(data_weather)):
    d = datetime.strptime(str(data_weather["YEARMODA"][i]), '%Y%m%d')
    dates_weather.append(d)

dates_weather = pd.DataFrame(dates_weather)
data_weather = pd.concat([dates_weather, data_weather], axis = 1)
print data_weather.columns.values
data_weather = data_weather.drop("YEARMODA", axis=1)
print data_weather.head()


[0 'YEARMODA' 'TEMP' 'PRCP']
           0  TEMP PRCP
0 2014-01-01  49.1    0
1 2014-01-02  47.5    0
2 2014-01-03  40.6    0
3 2014-01-04  53.8    0
4 2014-01-05  51.8    0


In [6]:
data_weather = data_weather.rename(index=str, columns = {0:"Date"})
print data_weather.head()
data_weather.to_csv("WeatherDataClean.csv")

        Date  TEMP PRCP
0 2014-01-01  49.1    0
1 2014-01-02  47.5    0
2 2014-01-03  40.6    0
3 2014-01-04  53.8    0
4 2014-01-05  51.8    0


In [7]:
file_2014 = "items-2014-01-01-2015-01-01.csv"
file_2015 = "items-2015-01-01-2016-01-01.csv"
file_2016 = "items-2016-01-01-2017-01-01.csv"

data_2014 = pd.read_csv(file_2014)
print(data_2014.shape)

data_2015 = pd.read_csv(file_2015)
print(data_2015.shape)

data_2016 = pd.read_csv(file_2016)
print(data_2016.shape)

data_Full = pd.concat([data_2014, data_2015, data_2016], ignore_index=True)

features = ["Date", "Time", "Category", "Item", "Qty", "Price Point Name", "Net Sales", "Transaction ID"]
data = data_Full[features]
print(data.shape)
for ft in features:
    print(type(data_Full.loc[1,ft]))

(108654, 24)


/Users/michaelspearing/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(147186, 24)
(190065, 14)
(445905, 8)
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'numpy.int64'>
<type 'str'>
<type 'str'>
<type 'str'>


In [8]:
# Lets take a look at all of the categories per feature.
features_unique_count = {}
features_unique = {}
for feature in features:
    features_unique[feature] = data[feature].unique()
    print(feature + " has %d unique values." %(features_unique[feature].size))

Date has 1059 unique values.
Time has 42141 unique values.
Category has 20 unique values.
Item has 243 unique values.
Qty has 35 unique values.
Price Point Name has 454 unique values.
Net Sales has 1302 unique values.
Transaction ID has 239959 unique values.


In [9]:
print ("START TIME: " + str(time.asctime( time.localtime(time.time()))))
# Add some features that correspond to time
# Month
dates = [None]*len(data)
times = [None]*len(data)
month = [None]*len(data)
year = [None]*len(data)
day_of_week = [None]*len(data)
day_of_month = [None]*len(data)
for i in range(len(data)):
    date = datetime.strptime((data.loc[i,"Date"]), '%m/%d/%Y')
    dates[i] = date
    t = datetime.strptime((data.loc[i,"Time"]), '%H:%M:%S')
    times[i] = t
    month[i] = (date.month)
    day_of_month[i] = (date.day)
    year[i] = date.year

    # Monday is 0, sunday is 6
    day_of_week[i] = date.weekday()
    if i % 100000 == 0:
        print("PROGRESS: %d" %i)
# Day of the week
# Day of the month
print ("END TIME: " + str(time.asctime( time.localtime(time.time()))))

START TIME: Tue May  2 16:49:04 2017
PROGRESS: 0
PROGRESS: 100000
PROGRESS: 200000
PROGRESS: 300000
PROGRESS: 400000
END TIME: Tue May  2 16:53:50 2017


In [10]:
# Add in time/date data
data = data.drop("Date", axis = 1)
data = data.drop("Time", axis = 1)
data.insert(0,"Date",pd.Series(dates))
data.insert(1,"Month",pd.Series(month))
data.insert(2,"Day_of_month", pd.Series(day_of_month))
data.insert(3,"Day_of_week", pd.Series(day_of_week))
data.insert(4, "Time", pd.Series(times))
data.insert(5,"Year",pd.Series(year))
print data.head()

        Date  Month  Day_of_month  Day_of_week                Time  Year  \
0 2014-12-31     12            31            2 1900-01-01 14:06:56  2014   
1 2014-12-31     12            31            2 1900-01-01 14:03:29  2014   
2 2014-12-31     12            31            2 1900-01-01 14:02:26  2014   
3 2014-12-31     12            31            2 1900-01-01 14:01:23  2014   
4 2014-12-31     12            31            2 1900-01-01 14:00:56  2014   

        Category         Item  Qty Price Point Name Net Sales  \
0   1 Hot Drinks  Caffe Latte    1            16 oz    $3.75    
1  2 Cold Drinks       Frappé    1    16.oz Caramel    $4.29    
2   1 Hot Drinks        Mocha    1            12 oz    $4.25    
3   1 Hot Drinks    Americano    1            12 oz    $2.29    
4   1 Hot Drinks   Cappuccino    1    Regular Price    $2.85    

                         Transaction ID  
0  aa80a203-88c8-4dae-a5e2-5f885e01182a  
1  926342ef-515a-4d48-96ac-e8c4e37b8590  
2  4f0f7023-cdd4-4b45-86d4

In [11]:
# Add in the weather data
data = (pd.DataFrame.merge(data, data_weather, on="Date"))
data.head()

,Date,Month,Day_of_month,Day_of_week,Time,Year,Category,Item,Qty,Price Point Name,Net Sales,Transaction ID,TEMP,PRCP
0,2014-12-31,12,31,2,1900-01-01 14:06:56,2014,1 Hot Drinks,Caffe Latte,1,16 oz,$3.75,aa80a203-88c8-4dae-a5e2-5f885e01182a,39.1,0
1,2014-12-31,12,31,2,1900-01-01 14:03:29,2014,2 Cold Drinks,Frappé,1,16.oz Caramel,$4.29,926342ef-515a-4d48-96ac-e8c4e37b8590,39.1,0
2,2014-12-31,12,31,2,1900-01-01 14:02:26,2014,1 Hot Drinks,Mocha,1,12 oz,$4.25,4f0f7023-cdd4-4b45-86d4-8f33893e8a25,39.1,0
3,2014-12-31,12,31,2,1900-01-01 14:01:23,2014,1 Hot Drinks,Americano,1,12 oz,$2.29,e31651d8-8543-467a-860b-cf890afeea7e,39.1,0
4,2014-12-31,12,31,2,1900-01-01 14:00:56,2014,1 Hot Drinks,Cappuccino,1,Regular Price,$2.85,3dcdc1ac-11b4-4657-9c4d-b26b99202856,39.1,0


In [12]:
# Strip off the $ symbol from the net Sales
print ("START TIME: " + str(time.asctime( time.localtime(time.time()))))
sales = [None]*len(data)
for i in range(len(data)):
    if(data.loc[i,"Net Sales"][0] == "("):
        tmp = np.float64(data.loc[i,"Net Sales"][2:-1])
        tmp = tmp * -1
    else:
        tmp = np.float64(data.loc[i,"Net Sales"][1:])
    sales[i] = tmp
    if i % 10000 == 0:
        print("PROGRESS: %d" %i)

START TIME: Tue May  2 16:53:50 2017
PROGRESS: 0
PROGRESS: 10000
PROGRESS: 20000
PROGRESS: 30000
PROGRESS: 40000
PROGRESS: 50000
PROGRESS: 60000
PROGRESS: 70000
PROGRESS: 80000
PROGRESS: 90000
PROGRESS: 100000
PROGRESS: 110000
PROGRESS: 120000
PROGRESS: 130000
PROGRESS: 140000
PROGRESS: 150000
PROGRESS: 160000
PROGRESS: 170000
PROGRESS: 180000
PROGRESS: 190000
PROGRESS: 200000
PROGRESS: 210000
PROGRESS: 220000
PROGRESS: 230000
PROGRESS: 240000
PROGRESS: 250000
PROGRESS: 260000
PROGRESS: 270000
PROGRESS: 280000
PROGRESS: 290000
PROGRESS: 300000
PROGRESS: 310000
PROGRESS: 320000
PROGRESS: 330000
PROGRESS: 340000
PROGRESS: 350000
PROGRESS: 360000
PROGRESS: 370000
PROGRESS: 380000
PROGRESS: 390000
PROGRESS: 400000
PROGRESS: 410000
PROGRESS: 420000
PROGRESS: 430000
PROGRESS: 440000


In [13]:
data = data.drop("Net Sales", axis=1)
data.insert(9, "Net Sales", pd.Series(sales))
print(data.head())

        Date  Month  Day_of_month  Day_of_week                Time  Year  \
0 2014-12-31     12            31            2 1900-01-01 14:06:56  2014   
1 2014-12-31     12            31            2 1900-01-01 14:03:29  2014   
2 2014-12-31     12            31            2 1900-01-01 14:02:26  2014   
3 2014-12-31     12            31            2 1900-01-01 14:01:23  2014   
4 2014-12-31     12            31            2 1900-01-01 14:00:56  2014   

        Category         Item  Qty  Net Sales Price Point Name  \
0   1 Hot Drinks  Caffe Latte    1       3.75            16 oz   
1  2 Cold Drinks       Frappé    1       4.29    16.oz Caramel   
2   1 Hot Drinks        Mocha    1       4.25            12 oz   
3   1 Hot Drinks    Americano    1       2.29            12 oz   
4   1 Hot Drinks   Cappuccino    1       2.85    Regular Price   

                         Transaction ID  TEMP PRCP  
0  aa80a203-88c8-4dae-a5e2-5f885e01182a  39.1    0  
1  926342ef-515a-4d48-96ac-e8c4e37b8590 

In [14]:
data.to_csv("items-2014-01-01-2017-01-01-Weather.csv")
alert(3)